Manipulators are verbs

In [ ]:
class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [ ]:
class Document {
  def write(content: InputStream): Unit = {
    // Implementation here
  }
}